
#### 点击上方菜单栏的“代码执行程序”，在“更改运行时类型”中选择硬件加速器为GPU
#### 避免Colab断开连接：按Ctrl+Shift+I等待弹窗然后点击控制台，输入以下内容并回车：


```
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()}
setInterval(ConnectButton,60000);
```



### 01 安装需要的依赖和部署预训练模型

In [ ]:
!python -m pip install paddlepaddle-gpu==2.4.1.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!python -m pip install ppasr
!python -m pip install paddlespeech-ctcdecoders
!pip install pypinyin
!sudo apt-get install sox
!git clone https://github.com/AlexandaJerry/PPASR.git
%cd /content/PPASR
!python setup.py install

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN" -O "PPASR.zip" && rm -rf /tmp/cookies.txt
!unzip /content/PPASR/PPASR.zip -d /content
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/configs/ /content/PPASR/configs/
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/dataset/ /content/PPASR/dataset/
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/models/ /content/PPASR/models/
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1--Sy7fcquKBR8PNIVW-quDlSD1fCNcGk' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1--Sy7fcquKBR8PNIVW-quDlSD1fCNcGk" -O "zh_giga.no_cna_cmn.prune01244.klm" && rm -rf /tmp/cookies.txt
!cp -r /content/PPASR/zh_giga.no_cna_cmn.prune01244.klm /content/PPASR/lm/
!python export_model.py --resume_model=models/conformer_online_fbank/best_model/



---



---



### 02 导入谷歌云盘中的音频压缩文件
#### 谷歌云盘内音频压缩包的文件名填入wav_file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
wav_file = '2023_02_11.zip'#@param {type:"string"}
wav_path = "/content/drive/MyDrive/" + wav_file
!unzip $wav_path -d /content/PPASR/raw_wav

In [ ]:
import glob
file_list = glob.glob('/content/PPASR/raw_wav/**/*.wav',recursive=True)
file_list.sort()
for i in range(len(file_list)):
  print(file_list[i])



---



---



### 04 切片后的短音频自动转写和转拼音

In [13]:
!python infer_wav.py

初始化解码器...
language model: model path = lm/zh_giga.no_cna_cmn.prune01244.klm, is_character_based = True, max_order = 5, dict_size = 0
初始化解码器完成!
[2023-02-11 13:29:08.320462 INFO   ] inference_predictor:__init__:71 - 已加载模型：models/conformer_online_fbank/infer/
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0001.wav,消耗时间：856ms,识别结果: 朋友是律师
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0002.wav,消耗时间：180ms,识别结果: 爸爸喜欢看书
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0003.wav,消耗时间：162ms,识别结果: 哥哥是医生
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0004.wav,消耗时间：189ms,识别结果: 我喜欢看电视
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0005.wav,消耗时间：145ms,识别结果: 他们是护士
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0006.wav,消耗时间：268ms,识别结果: 高鹏和许涛是邻居
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0007.wav,消耗时间：171ms,识别结果: 他不叫高鹏
正在识别：/content/PPASR/raw_wav/hp0317_chn_t0_m/hp0317_chn_t0_m_0008.wav,消耗时间：183ms,识别结果: 弟弟喜欢跳舞
正

In [ ]:
import os
import sys
import numpy as np
from pypinyin import pinyin, lazy_pinyin, Style
import re

file_list = glob.glob('/content/PPASR/raw_wav/**/*.txt',recursive=True)
file_list.sort()

r="[_.!+-=——,$%^,。?、~@#￥%……&*《》<>「」{}【】()（）/''\n， ]"

for file in file_list:
			text_file = open(file)
			line = text_file.read().strip()
			line = re.sub(r,"",line)
			pinyin =  lazy_pinyin(line, style=Style.TONE3, errors='default', strict=False, v_to_u=False, neutral_tone_with_five=True,tone_sandhi=True)
			pinyinline = ' '.join(pinyin)
			# print(line)
			target_text_file = open(file, "w")
			target_text_file.write(pinyinline)
			target_text_file.close()

In [19]:
## 加载MFA普通话词典检测有无超出词典范围的词出现
import tqdm
dict_path = '/content/PPASR/mandarin_pinyin.txt'
with open(dict_path, 'r', encoding='utf8') as f:
    rules = [ln.strip().split('\t') for ln in f.readlines()]
dictionary = {}
phoneme_set = set()
for r in rules:
    phonemes = r[1].split()
    dictionary[r[0]] = phonemes
    phoneme_set.update(phonemes)

# Run checks
check_failed = False
covered = set()
phoneme_map = {}
for ph in sorted(phoneme_set):
    phoneme_map[ph] = 0

segment_pairs = []

sliced_path = '/content/PPASR/raw_wav/'
sliced_filelist=glob.glob('/content/PPASR/raw_wav/**/*.wav',recursive=True)

for file in tqdm.tqdm(sliced_filelist):
    annotation = file.replace(".wav",".txt")
    if not os.path.exists(annotation):
        print(f'No annotation found for \'{annotation}\'!')
        check_failed = True
    with open(annotation, 'r', encoding='utf8') as f:
        syllables = f.read().strip().split()
    if not syllables:
        print(f'Annotation file \'{annotation}\' is empty!')
        check_failed = True
    else:
        oov = []
        for s in syllables:
            if s not in dictionary:
                oov.append(s)
            else:
                for ph in dictionary[s]:
                    phoneme_map[ph] += 1
                covered.update(dictionary[s])
        if oov:
            print(f'Syllable(s) {oov} not allowed in annotation file \'{annotation}\'')
            check_failed = True

100%|██████████| 417/417 [00:00<00:00, 27107.42it/s]


### 05 安装MFA和进行对齐

In [23]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()
!conda env create -n aligners -f /content/PPASR/environment.yml

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
✨🍰✨ Everything looks OK!
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c default

In [24]:
!source activate aligners; \
mfa version

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/usr/local/envs/aligners/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
2.0.6


In [29]:
# Slicer arguments
beam_number = 100 #@param {type:"number"}
textgrid_dir = 'outputs'#@param {type:"string"}
textgrid_path = '/content/PPASR/' + textgrid_dir
%mkdir $textgrid_path
wav_path = '/content/PPASR/raw_wav'
acoustic_model = '/content/PPASR/mandarin.zip'
dictionary = '/content/PPASR/mandarin_pinyin.txt'
acoustic_model = '/content/PPASR/mandarin.zip'
dictionary = '/content/PPASR/mandarin_pinyin.txt'


!source activate aligners; \
mfa align --clean $wav_path $dictionary $acoustic_model $textgrid_path --beam $beam_number --overwrite

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: cannot create directory ‘/content/PPASR/outputs’: File exists
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/usr/local/envs/aligners/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
INFO - Setting up corpus information...
INFO - Loading corpus from source files...
100% 417/417 [00:01<00:00, 312.57it/s] 
INFO - Found 10 speakers across 417 files, average number of utterances per speaker: 41.7
INFO - Initializing multiprocessing jobs...
INFO - Creating corpus split for feature generation...
INFO - Generating base features (mfcc).

### 06 保存短音频和对应转写和对齐文件到谷歌云盘
#### 注意保存文件名不要与已有文件名相同 否则可能提示覆盖谷歌云盘的文件

In [30]:
from google.colab import files
%cd /content/PPASR/raw_wav
!cp -RT /content/PPASR/outputs /content/PPASR/raw_wav
zip_name_you_wish = 'transcription.zip'#@param {type:"string"}
zip_path = zip_name_you_wish
!zip -r $zip_path './' -x '*.wav'
# !cp -i $zip_path "/content/drive/MyDrive/"
# print("files have been saved to MyDrive")
print("files would be automatically downloaded after several seconds")
files.download(zip_path)

/content/PPASR/raw_wav
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  adding: sh1109_chn_t0_m/ (stored 0%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0004.TextGrid (deflated 84%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0039.txt (deflated 3%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0023.txt (stored 0%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0037.txt (stored 0%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0038.txt (deflated 12%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0017.txt (deflated 9%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0041.TextGrid (deflated 86%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0038.TextGrid (deflated 85%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0026.txt (stored 0%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0041.txt (deflated 4%)
  adding: sh1109_chn_t0_m/sh1109_chn_t0_m_0031.txt (stored 0%)
  adding: sh1109

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>